# Shifting Db2 with HADR
One option when shifting a database is to make the new copy an HADR standby database. Once comfortable with the operation of the Db2 copy, the standby can be switched to take on the primary role. This allows for a phased migration of the database and the workloads.

To set up an HADR pair with Db2 Shift requires two steps:
* Shift the primary database as an HADR clone
* Initialize the primary and secondary systems

All the shift and deploy menus include an option that determines if the database will be deployed as an HADR secondary. When this option is selected, the shift will be performed and then the database will be placed into an "HADR" mode waiting for the second step which connects the primary to the secondary.

This example also demonstrates how we can create a Db2 database at another instance. The Db2 Shift command can take an on-premise Db2 database and shift it into a traditional instance that is running on the Cloud (EC2 instance), a VMWare server or any type of virtualize environment. As long as the program can access the server via SSH, the shift can be done.

## Terminal Window
Before running the examples in this lab, you will need to open up a terminal window. If you have not already done so, see the <a href="./Terminal.ipynb">Opening up a Terminal Window</a> section. 

The commands that you need to copy and execute in the terminal window are surrounded with a blue box. Select all the text in the box and then use the copy keys (^C or &#x2318;C) to place the text into the clipboard.

<div class="boxed">The command you need to copy and paste into the terminal window are found in here.
</div>

When you switch to the terminal window, you will use the appropriate Paste command (^&#x21E7;v or &#x2318;v). This will load the command into the terminal window and immediately execute it.

## Initialize Primary Database with Logging
To use HADR you must ensure that the following two settings have been updated for the database:
* LOGARCHMETH1 – Log archive method and location
* LOGINDEXBUILD – Needs to be set on for faster recovery

The `TRADING` database was not created with these settings enabled so you will need to make the changes before attempting to shift the database. This step is crucial because the target database must also include these settings so you must have them in place  on the source database before shifting the database.

In your terminal window, connect to `db2inst1` on server 1 (`10.0.0.1`).

<div class="boxed">ssh db2inst1@10.0.0.1</div>

As db2inst1@server1, create a logfile directory and then update the two settings for the `TRADING` database.

<div class="boxed">mkdir ~/logfiles
db2 "update db config for trading using logarchmeth1 DISK:/home/db2inst1/logfiles"
db2 "update db config for trading using logindexbuild on"
</div>

We are not done at this point. Since we have changed the logging for the database, a full backup is required for the change to take effect. The following command will backup the database (but don't ever use this technique in a production environment!)

<div class="boxed">db2 "connect reset"
db2 "backup database trading to /dev/null"
</div>

The command will return with a successful backup.
```
Backup successful. The timestamp for this backup image is : 20220614093844
```
The database is now ready to be shifted to server 2 and be set up to participate in an HADR pair.

## Add Read Only Capabilities
By default, the HADR secondary will not allow connections from clients. If you want to allow for read-only access, you must set the `DB2_HADR_ROS` environment variable at the source to `ON`. From a licensing perspective, you will need to include all the VPCs used on the target site if you intend to use the read-only feature.

As db2inst1@server1, run the following command to change the ROS setting.

<div class="boxed">db2set DB2_HADR_ROS=ON
db2set 
db2stop force
db2start
</div>

## Open Up HADR Ports
HADR requires that port 3700 is open on both the source and target servers. If the ports are not open the HADR process will not be able to start!

As db2inst1@server1, open port 3700 and refresh the rules. **Note:** The ports may already be enabled.

<div class="boxed">sudo firewall-cmd --zone=public --add-port=3700/tcp --permanent
sudo firewall-cmd --reload
</div>

As db2inst1@server2, open port 3700 and refresh the rules.

<div class="boxed">ssh db2inst1@10.0.0.2
sudo firewall-cmd --zone=public --add-port=3700/tcp --permanent
sudo firewall-cmd --reload
exit
</div>

You should see `Connection to 10.0.0.2 closed.`

## Gather Shift Details
The target database is also called TRADING and you need to make sure that database creation is FORCED since the database does not currently exist on that server.

The owner of the database will be db2inst1@server2. Since the instance owner is the same between the two servers, you do not need to worry about adding db2inst1 as a SECADM or DBADM to the database.

The HADR option must also be used during this shift to place the target database into ACS standby mode which is required to use HADR.

![HADR](images/c2c-hadr.jpg)
 
The information you need is found below.
* Name of the local database (`TRADING`)
* Instance owner at target (`db2inst1`)
* The target database name (`TRADING`)
* Select `FORCE` database creation since we haven’t created a database
* Instance owner at target (`db2inst1`)
* Enter the address of the target server (`10.0.0.2`)

## Start Db2 Shift 
This step will shift the TRADING database to server 2 to use as part of an HADR pair. This will be a Db2-to-Db2 instance shift with the TRADING database being created at the target. With the db2inst1@server1 userid, start up the Db2 Shift command inside the c2c directory.

<div class="boxed">cd ~/c2c
./db2shift --shiftdb2
</div>

You should see the following screen.

![Db2 Shift](images/c2c-shiftinstance.jpg)
 
Update the information on this screen with the values you collected in step 4.
* Database Name: `TRADING`
* Instance Owner: `db2inst1`
* Target Database: `TRADING`
* Force Database Creation: `ON`
* Target Owner: `db2inst1`
* Target Server: `10.0.0.2`
* HADR Option: `ON`

The HADR option is used to create the database as an HADR secondary. Once you shift the database with this option, you will need to use the HADR setup menu to connect the primary and secondary databases.

The other options should be set to:
* Metadata: `Refresh`
* Sync: `None`
* HADR option: `ON`
* Exclude Routines: `OFF`
* Database Mode: `Online`
* Thread Count: `4`
* Compression: `0`
* Overrides: `None`

Once you have updated all the settings, press ^X to review the command and then ^X to execute the command. Once the shift has completed successfully, you will go onto the next step to connect the primary and secondary servers. Note that this shift will take longer since the `TRADING` database needs to be created at the second server before we can shift the database.

![HADR ACS](images/c2c-acs-standby.jpg)

The completion message indicates that the database has been started in mode ACS standby, waiting to connect to the primary server. Exit Db2 Shift when done.

## Connecting HADR Primary and Standby Servers

Up to this point in the lab you have successfully created the secondary server for the TRADING database. However, we have not yet connected the primary and secondary servers or started the HADR systems.

To setup up the connection between two servers, you will need to use the HADR Initialization menu. You can find this menu on the main Db2 Shift panel or access this panel directly. On db2inst1@server1 issue the following command:

<div class="boxed">cd ~/c2c
./db2shift --hadrdb2
</div>

The screen is shown below.

![HADR](images/c2c-hadr-menu.jpg)



The panel requires the following information:
* The source database name and server
* The destination server details

The Initialize HADR screen is used to initialize and start HADR between a source and target server. This step would be run after the database has been shifted to the new location. This option is not used for the initial HADR setup of the target system. The target database must be created with the HADR option for it be placed into the correct mode for HADR communication. 
 

### HADR Source and Destination Server
  
For destinations that are traditional Db2 instances, you must provide the IP address or symbolic name for both servers. You must use the full IP address for the source and destination servers so localhost would not be valid on this panel.

### HADR ports
  
HADR communicates over a port which is different than the Db2 instance. You must supply the source and destination port numbers that Db2 will communicate between the HADR servers. The default port number is 3700 for HADR communications on both the source and target ports.

## Gather Shift Details
Update the information on this screen with the following values.
* Database Name: `TRADING`
* Source HADR IP: `10.0.0.1`
* Source HADR Port: `3700`
* Target Owner: `db2inst1`
* Target HADR IP: `10.0.0.2`
* Target HADR Port: `3700`

## Running the Shift
Once you have filled in the information for the source and destination HADR databases, press ^X to review the command and then ^X to execute the code. As the process runs, you will see HADR settings being updated.

![HADR Running](images/c2c-hadr-running.jpg)
 
When the process completes you should see the success screen.

![HADR Success](images/c2c-hadr-success.jpg)

To double check that the HADR connection is complete, you can use the db2pd command to determine the status of HADR on both the source and target servers. Exit the Db2 Shift menu.
As db2inst1@server1, check the HADR status.

<div class="boxed">db2pd -db trading -hadr | head</div>

The output should be similar to:
```
Database Member 0 -- Database TRADING -- Active -- ...
             HADR_ROLE = PRIMARY
           REPLAY_TYPE = PHYSICAL
         HADR_SYNCMODE = NEARSYNC
            STANDBY_ID = 1
         LOG_STREAM_ID = 0
            HADR_STATE = PEER
            HADR_FLAGS = TCP_PROTOCOL
```

As db2inst1@server2, check the HADR status

<div class="boxed">ssh db2inst1@10.0.0.2
db2pd -db trading -hadr | head
exit
</div>

The output should be similar to:
```
Database Member 0 -- Database TRADING -- Active -- ...
             HADR_ROLE = PRIMARY
           REPLAY_TYPE = PHYSICAL
         HADR_SYNCMODE = NEARSYNC
            STANDBY_ID = 1
         LOG_STREAM_ID = 0
            HADR_STATE = PEER
            HADR_FLAGS = TCP_PROTOCOL
```

To make sure that the HADR process is working properly, make the following change to the `TRADING` database on the source.

As db2inst1@server1, create a new table based on the CUSTOMER table.

<div class="boxed">db2 "connect to trading"
db2 "create table custcopy as (select * from trading.customers) with data"
db2 "select count(*) from custcopy"
</div>

There should be 10,000 rows in the custcopy table. As db2inst1@server2, check the contents of the TRADING table.

<div class="boxed">ssh db2inst1@10.0.0.2
db2 "connect to trading"
db2 "select count(*) from custcopy with ur"
exit
</div>

Note the use of `with ur` for the query to work. Hopefully they are the same count of 10,000 rows!

## Summary
In this chapter you:
* Set up a database so that it can be used in an HADR environment
* Learned how shift a database that will be used as part of an HADR pair
* Connected the two servers
* Verified that the two servers are communicating with one another 

#### Credits: IBM 2024, George Baklarz [baklarz@ca.ibm.com]